<img src="../images/QISKit-c.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="250 px" align="left">

## _*VQE algorithm: Application to quantum chemistry*_ 

The latest version of this notebook is available on https://github.com/IBM/qiskit-sdk-py/tree/master/scripts.

For more information about how to use the IBM Q Experience (QX), consult the [Quantum Experience tutorials](https://quantumexperience.ng.bluemix.net/qstage/#/tutorial?sectionId=c59b3710b928891a1420190148a72cce&pageIndex=0), or check out the [community](https://quantumexperience.ng.bluemix.net/qstage/#/community).

***
### Contributors
Antonio Mezzacapo, Jay Gambetta

### Introduction

One of the earliest and most compelling applications of quantum computation is the idea of using quantum computers to simulate other quantum systems. Quantum simulation of quantum systems encompasses a wide range of tasks, including most significantly:
    
1. Simulation of the time evolution of quantum systems.

2. Computation of ground state properties. 

These applications are especially useful when considering systems of interacting fermions, such as molecules and strongly correlated materials. The computation of ground state properties of fermionic systems is the starting point for mapping out the phase diagram of condensed matter Hamiltonians. It also gives access to the key question of electronic structure problems in quantum chemistry, namely reaction rates. The focus of this notebook is on molecular systems, which are considered to be the ideal bench test for early-stage quantum computers, because of their relevance in chemical applications despite relatively modest sizes. Formally, the ground state problem asks the following:

For some physical Hamiltonian *H*, find the smallest eigenvalue $E_G$, such that $H|\psi_G\rangle=E_G|\psi_G\rangle$, where $|\Psi_G\rangle$ is the eigenvector corresponding to $E_G$.
    
It is known that in general this problem is intractable even on a quantum computer. This means that we cannot expect an efficient quantum algorithm which prepares the ground state of general local Hamiltonians. Despite this limitation, for specific Hamiltonians of interest it might be possible, given physical constraints on the interactions, to solve the above problem efficiently. Currently, at least four different methods exist to approach this problem:

1. Quantum phase estimation: Assuming that we can approximately prepare the state $|\psi_G\rangle$, this routine uses controlled implementations of the Hamiltonian to find its smallest eigenvalue. 
2. Adiabatic theorem of quantum mechanics: The quantum system is adiabatically dragged from being the ground state of a trivial Hamiltonian to the one of the target problem, via slow modulation of the Hamiltonian terms. 
3. Dissipative (non-unitary) quantum operation: It has the ground state of the target system as a fixed point. The non-trivial assumption here is the implementation of the dissipation map on quantum hardware. 
4. Variational quantum eigensolvers: Here we assume that ground state can be represented by a parameterization containing a relatively small number of parameters.

In this notebook we focus on the last method, as this is most likely the simplest to be realized on near-term devices. 

The general idea is to define a parameterization $|\psi(\boldsymbol\theta)\rangle$ of quantum states, and minimize the energy 

$$E(\boldsymbol\theta) = \langle \psi(\boldsymbol\theta)| H |\psi(\boldsymbol\theta)\rangle,$$ 

The key ansatz is that the number of parameters $|\boldsymbol\theta^*|$ that minimizes the energy function scales polynomially with the size (e.g., number of qubits) of the target problem.  

Then, any local fermionic Hamiltonian can be mapped into a sum over pauli operators $P_i$,  

$$H\rightarrow H_P = \sum_i^M w_i P_i,$$ 

and the energy corresponding to the state $|\psi(\boldsymbol\theta\rangle$, $E(\boldsymbol\theta)$, can be estimated by sampling the individual Pauli terms $P_i$ (or sets of them that can be measured at the same time) on a quantum computer

$$E(\boldsymbol\theta) = \sum_i^M w_i \langle \psi(\boldsymbol\theta)| P_i |\psi(\boldsymbol\theta)\rangle.$$ 

Last, some optimization technique has to be devised in order to find the optimal value of parameters $\boldsymbol\theta^*$, such that $|\psi(\boldsymbol\theta^*)\rangle\equiv|\psi_G\rangle$.



### Fermionic Hamiltonians

The Hamiltonians describing systems of interacting fermions can be expressed in second quantization language, considering fermionic creation (annihilation) operators $a^\dagger_\alpha(a_\alpha)$, relative to the $\alpha$-th fermionic mode. In the case of molecules, the $\alpha$ labels stand for the different atomic or molecular orbitals. Within the second-quantization framework, a generic molecular Hamiltonian with $M$ orbitals can be written as 
$$H =H_1+H_2=\sum_{\alpha, \beta=0}^{M-1} t_{\alpha \beta} \, a^\dagger_{\alpha} a_{\beta} +\frac{1}{2}  \sum_{\alpha, \beta, \gamma, \delta = 0}^{M-1} u_{\alpha \beta \gamma \delta}\, a^\dagger_{\alpha} a^\dagger_{\gamma} a_{\delta} a_{\beta},$$
with the one-body terms representing the kinetic energy of the electrons and the potential energy that they experience in the presence of the nuclei, 
$$ t_{\alpha\beta}=\int d\boldsymbol x_1\Psi_\alpha(\boldsymbol{x}_1) \left(-\frac{\boldsymbol\nabla_1^2}{2}+\sum_{i} \frac{Z_i}{|\boldsymbol{r}_{1i}|}\right)\Psi_\beta (\boldsymbol{x}_1),$$
and their interactions via Coulomb forces 
$$ u_{\alpha\beta\gamma\delta}=\int\int d \boldsymbol{x}_1 d \boldsymbol{x}_2 \Psi_\alpha^*(\boldsymbol{x}_1)\Psi_\beta(\boldsymbol{x}_1)\frac{1}{|\boldsymbol{r}_{12}|}\Psi_\gamma^*(\boldsymbol{x}_2)\Psi_\delta(\boldsymbol{x}_2),$$
where we have defined the nuclei charges $Z_i$, the nuclei-electron and electron-electron separations $\boldsymbol{r}_{1i}$ and $\boldsymbol{r}_{12}$, the $\alpha$-th orbital wavefunction $\Psi_\alpha(\boldsymbol{x}_1)$, and we have assumed that the spin is conserved in the spin-orbital indices $\alpha,\beta$ and $\alpha,\beta,\gamma,\delta$.

### Molecules considered in this notebook and mapping to qubits

We consider in this notebook the optimization of two potential energy surfaces, for the hydrogen and lithium hydride molecules, obtained using the STO-3G basis. The molecular Hamiltonians are computed as a function of their interatomic distance and then mapped to 2 (H$_2$) and 4 (LiH$_2$) qubit problems, via elimination of core and high-energy orbitals and removal of $Z_2$ symmetries.

### Approximate Universal Quantum Computing for quantum chemisty problems

In order to find the optimal parameters $\boldsymbol\theta^*$, we set up a closed optimization loop with a quantum computer, based on some stochastic optimization routine. Our choice for the variational ansatz is a deformation of the one used for the optimization of classical combinatorial problems, with the inclusion of Z rotation together with the Y ones. The optimization algorithm for fermionic Hamiltonians is similar to the one for combinatorial problems, and can be summarized as follows: 


1. Map the fermionic Hamiltonian $H$ to a qubit Hamiltonian $H_P$.
2. Choose the maximum depth of the quantum circuit (this could be done adaptively).
3. Choose a set of controls $\boldsymbol\theta$ and make a trial function $|\psi(\boldsymbol\theta)\rangle$. The difference with the combinatorial problems is the insertion of additional parametrized Z single-qubit rotations.
4. Evaluate the energy $E(\boldsymbol\theta) = \langle\psi(\boldsymbol\theta)~|H_P|~\psi(\boldsymbol\theta)\rangle$ by sampling each Pauli term individually, or sets of Pauli terms that can be measured in the same tensor product basis.
5. Use a classical optimizer to choose a new set of controls.
6. Continue until the energy has converged, hopefully close to the real solution $\boldsymbol\theta^*$ and return the last value of $E(\boldsymbol\theta)$.
    

Note that, as opposed to the classical case, in the case of a quantum chemistry Hamiltonian one has to sample over non-computational states that are superpositions, and therefore take advantage in using a quantum computer in the sampling part of the algorithm. Motivated by the quantum nature of the answer, we also define a variational trial ansatz in this way, 

$$|\psi(\boldsymbol\theta)\rangle  = [U_\mathrm{single}(\boldsymbol\theta) U_\mathrm{entangler}]^m |+\rangle$$

where $U_\mathrm{entangler}$ is a collection of cPhase gates (fully entangling gates), $U_\mathrm{single}(\boldsymbol\theta) = \prod_{i=1}^n Y(\theta_{i})Z(\theta_{n+i})$ are single-qubit Y and Z rotation, $n$ is the number of qubits and $m$ is the depth of the quantum circuit. 



References and additional details:

[1] A. Kandala, A. Mezzacapo, K. Temme, M. Takita, J. M. Chow, and J. M. Gambetta, *Hardware-efficient Quantum Optimizer for Small Molecules and Quantum Magnets*, e-print arXiv 1704.0501, and references therein.


In [1]:
# Checking the version of PYTHON; we only support 3 at the moment
import sys
if sys.version_info < (3,0):
    raise Exception("Please use Python version 3 or greater.")
    
    
# useful additional packages 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy import linalg as la


import sys
sys.path.append("../../")
# importing the QISKit

from qiskit import QuantumCircuit, QuantumProgram
import Qconfig
import math

# import basic plot tools
from qiskit.basicplotter import plot_histogram

# import optimization tools
from tools.optimizationtools import trial_circuit_ryrz,trial_circuit_ry, SPSA_optimization, SPSA_calibration
from tools.optimizationtools import Energy_Estimate, Measure_pauli_z, Hamiltonian_from_file, make_Hamiltonian
from functools import partial
from tools.pauli import Pauli, label_to_pauli
from tools.fermion_to_qubit_tools import fermionic_maps, two_qubit_reduction



def eval_hamiltonian(Q_program,n,m,file_name,device,shots,H,theta):

    
    avg_energy=0
    var_energy=0
    std_energy=0

    # STACK QUANTUM CIRCUITS ASSOCIATED TO EACH PAULI TERM AND EXECUTE THEM 
    
    if shots!=1:
        
        circuits=[]
        trial_circuit=[]
        
        file = open(file_name, 'r+')
        ham_array = file.readlines()
        ham_array = [x.strip() for x in ham_array]
        
        for i in range(len(ham_array)//2):     

            trial_circuit.append(trial_circuit_ryrz(n,m,theta,entangler_map,ham_array[i*2],True))
            circuits.append("trial_circuit_label"+str(i))
            Q_program.add_circuit(circuits[i],trial_circuit[i])
   
        file.close()

        # EXECUTE THE CIRCUITS STACKED 

        Q_program.execute(circuits,device,shots,silent=True)
    
        pauli_list=Hamiltonian_from_file(file_name)

        for i in range(len(ham_array)//2):#loop over Pauli terms

            pauli_avg=Energy_Estimate(Q_program.get_counts("trial_circuit_label"+str(i)),pauli_list[i])
            avg_energy+=pauli_avg
    
    else:
    
        circuit=['circuit']
        Q_program.add_circuit(circuit[0],trial_circuit_ryrz(n,m,theta,entangler_map,None,False))
        Q_program.execute(circuit,device,shots,silent=True)
        avg_energy=(np.inner(np.conjugate(Q_program.get_data(circuit[0])['quantum_state']),np.dot(H,Q_program.get_data(circuit[0])['quantum_state']))).real
        
    
    return avg_energy,std_energy
                        

## Optimization of H2 at bond length

In this first part of the notebook we show the optimization of the H$_2$ Hamiltonian in the STO-3G basis at the bond length of 0.735 Angstrom. After mapping it to a 4-qubit system with a binary-tree transformation, two spin-parity symmetries are modded out, leading to a 2-qubit Hamiltonian. The energy of the mapped Hamiltonian obtained is then minimized using the variational ansatz described in the introduction, and a stochastic perturbation simultaneous approximation gradient descent method. 

In [ ]:
n=2
m=2
device='local_qasm_simulator'

initial_theta=np.zeros(2*n*m)
entangler_map={0: [1]} # the map of two-qubit gates with control at key and target at values
shots=1
max_trials=100
ham_name='H2/H2Equilibrium.txt'

#Exact Energy
pauli_list=Hamiltonian_from_file(ham_name)
H=make_Hamiltonian(pauli_list)
exact=np.amin(la.eig(H)[0]).real
        
    
# Optimization
Q_program = QuantumProgram()
Q_program.set_api(Qconfig.APItoken,Qconfig.config["url"])


eval_hamiltonian_partial=partial(eval_hamiltonian,Q_program,n,m,ham_name,device,shots,H)
eval_hamiltonian_partial(initial_theta)

initial_c=0.1
target_update=2*np.pi*0.1
SPSA_params=SPSA_calibration(eval_hamiltonian_partial,initial_theta,initial_c,target_update,25)
save_step=20

output=SPSA_optimization(eval_hamiltonian_partial,initial_theta,SPSA_params,max_trials,save_step,1);


plt.plot(np.arange(0, max_trials,save_step),output[2],label='E(theta_plus)')
plt.plot(np.arange(0, max_trials,save_step),output[3],label='E(theta_minus)')
plt.plot(np.arange(0, max_trials,save_step),np.ones(max_trials//save_step)*output[0],label='Final Energy')
plt.plot(np.arange(0, max_trials,save_step),np.ones(max_trials//save_step)*exact,label='Exact Energy')
plt.legend()
plt.xlabel('Trial state')
plt.ylabel('Energy')

ERROR in _quantumprogram._setup_api: __init__() takes from 2 to 3 positional arguments but 4 were given
calibration step # 0 of 25
calibration step # 5 of 25
calibration step # 10 of 25
calibration step # 15 of 25
calibration step # 20 of 25
calibrated SPSA_parameters[0] is 1.54362989662
objective function at theta+ for step # 0
-0.801795812412
objective function at theta- for step # 0
-0.951619232087
objective function at theta+ for step # 20
-1.80283241706
objective function at theta- for step # 20
-1.78487805242
objective function at theta+ for step # 40
-1.83104432525
objective function at theta- for step # 40
-1.83725040225
objective function at theta+ for step # 60
-1.85103193445
objective function at theta- for step # 60
-1.84968503118


In [7]:
print(np.zeros(initial_theta.shape))
print(np.arange(0,100,20))

[ 0.  0.  0.  0.  0.  0.  0.  0.]
[ 0 20 40 60 80]


## Optimizing the potential energy surface 

The optimization considered previously is now performed for two molecules, H$_2$ and LiH, for different interatomic distances, and the correspoding nuclei Coulomb repulsion is added in order to obtain a potential energy surface. 

In [ ]:
# MOLECULE PARAMETERS

molecule='LiH' #name of the molecule: options are H2 or LiH

if molecule=='H2':
    n=2 # qubits
    Z1=1
    Z2=1
    min_distance=.2
    max_distance=4
    number_of_points=39
elif molecule=='LiH':
    n=4 # qubits
    Z1=1
    Z2=3
    min_distance=.5
    max_distance=5
    number_of_points=46
    
    
# OPTIMIZATION PARAMETERS 

run_optimization=True # Choose True or False. False just plots the exact potential energy surface

m=2 # depth (number of layers of sq gates - # entanglers is this number -1)
device='local_qasm_simulator'
initial_theta=np.random.randn(2*n*m)  # initial angles 
entangler_map={0: [1]} # the map of two-qubit gates with control at key and target at values
shots=1
max_trials=500

initial_c=0.01
target_update=2*np.pi*0.1





####################     COMPUTING THE POTENTIAL ENERGY SURFACE      ###################

mol_distance=np.zeros(number_of_points)
coulomb_repulsion=np.zeros(number_of_points)
electr_energy=np.zeros(number_of_points)
electr_energy_optimized=np.zeros(number_of_points)

Q_program = QuantumProgram()
Q_program.set_api(Qconfig.APItoken,Qconfig.config["url"])

for i in range(number_of_points):

    
    # %%%%%%%%%%%%% Coulomb Repulsion For Diatomic molecules %%%%%%%%%%%%%%%%%%%%%%
        mol_distance[i]=np.around((min_distance+(max_distance-min_distance)*i/(number_of_points-1)),2)
        
        distance=mol_distance[i]/0.529177
        coulomb_repulsion[i]=Z1*Z2/distance
        
    # exact diagonalization
        ham_text=molecule+'/PESMap'+str(i)+'atdistance'+str(mol_distance[i])+'.txt'
        pauli_list=Hamiltonian_from_file(ham_text)
        H=make_Hamiltonian(pauli_list)
        eigen=la.eig(H)
        electr_energy[i]=np.amin(eigen[0])
           
    # optimization
    
        if run_optimization:
            print('\nOPTIMIZING HAMILTONIAN # '+str(i)+' AT INTERATOMIC DISTANCE '+str(mol_distance[i])+' ANGSTROM\n')
            eval_hamiltonian_partial=partial(eval_hamiltonian,Q_program,n,m,ham_name,device,shots,H)
            SPSA_params=SPSA_calibration(eval_hamiltonian_partial,initial_theta,initial_c,target_update,5)
            electr_energy_optimized[i]=SPSA_optimization(eval_hamiltonian_partial,initial_theta,SPSA_params,max_trials,5)[0]
            


plt.plot(mol_distance,electr_energy+coulomb_repulsion,label='Exact')
if run_optimization:
    plt.plot(mol_distance,electr_energy_optimized+coulomb_repulsion,label='Optimized') 
plt.xlabel('Atomic distance (Angstrom)')
plt.ylabel('Energy')
plt.legend()
        

/Users/amezzac/anaconda/envs/QISKitenv/lib/python3.6/site-packages/ipykernel_launcher.py:63: ComplexWarning: Casting complex values to real discards the imaginary part



OPTIMIZING HAMILTONIAN # 0 AT INTERATOMIC DISTANCE 0.5 ANGSTROM

calibration step # 0 of 5
calibrated SPSA_parameters[0] is 2.00109757451
objective function at theta+ for step # 0
-9.41047661529
objective function at theta- for step # 0
-9.40477009492
objective function at theta+ for step # 5
-9.11929360427
objective function at theta- for step # 5
-9.12473560924
objective function at theta+ for step # 10
-9.37869272122
objective function at theta- for step # 10
-9.37793476068
objective function at theta+ for step # 15
-9.5856162279
objective function at theta- for step # 15
-9.59240571774
objective function at theta+ for step # 20
-9.78506685288
objective function at theta- for step # 20
-9.79176811501
objective function at theta+ for step # 25
-10.0232854719
objective function at theta- for step # 25
-10.0218747772
objective function at theta+ for step # 30
-10.0451112552
objective function at theta- for step # 30
-10.0452595283
objective function at theta+ for step # 35
-10.0557464

objective function at theta+ for step # 345
-10.096347146
objective function at theta- for step # 345
-10.0964513122
objective function at theta+ for step # 350
-10.0963957477
objective function at theta- for step # 350
-10.0965099967
objective function at theta+ for step # 355
-10.0965729378
objective function at theta- for step # 355
-10.0964126386
objective function at theta+ for step # 360
-10.0966536921
objective function at theta- for step # 360
-10.0963549657
objective function at theta+ for step # 365
-10.096467656
objective function at theta- for step # 365
-10.0966237698
objective function at theta+ for step # 370
-10.0967215344
objective function at theta- for step # 370
-10.0964911789
objective function at theta+ for step # 375
-10.0968441568
objective function at theta- for step # 375
-10.0964142203
objective function at theta+ for step # 380
-10.0966801294
objective function at theta- for step # 380
-10.0968398348
objective function at theta+ for step # 385
-10.0966628832

objective function at theta+ for step # 190
-9.79150252508
objective function at theta- for step # 190
-9.79124426908
objective function at theta+ for step # 195
-9.79148338832
objective function at theta- for step # 195
-9.79153796744
objective function at theta+ for step # 200
-9.79149040359
objective function at theta- for step # 200
-9.79152538176
objective function at theta+ for step # 205
-9.79155610933
objective function at theta- for step # 205
-9.79163567859
objective function at theta+ for step # 210
-9.79174482117
objective function at theta- for step # 210
-9.79156875211
objective function at theta+ for step # 215
-9.79177144386
objective function at theta- for step # 215
-9.79163171469
objective function at theta+ for step # 220
-9.79173055256
objective function at theta- for step # 220
-9.79167714553
objective function at theta+ for step # 225
-9.79179429347
objective function at theta- for step # 225
-9.79182841435
objective function at theta+ for step # 230
-9.791856724

objective function at theta+ for step # 35
-9.58501170836
objective function at theta- for step # 35
-9.58256786026
objective function at theta+ for step # 40
-9.58999360023
objective function at theta- for step # 40
-9.59096625464
objective function at theta+ for step # 45
-9.59791073063
objective function at theta- for step # 45
-9.59684406328
objective function at theta+ for step # 50
-9.6001594532
objective function at theta- for step # 50
-9.59967902577
objective function at theta+ for step # 55
-9.60172466439
objective function at theta- for step # 55
-9.60216846157
objective function at theta+ for step # 60
-9.60312178891
objective function at theta- for step # 60
-9.6026878965
objective function at theta+ for step # 65
-9.60323875683
objective function at theta- for step # 65
-9.60385193655
objective function at theta+ for step # 70
-9.60409215458
objective function at theta- for step # 70
-9.60411076406
objective function at theta+ for step # 75
-9.60578848794
objective functi

...

objective function at theta+ for step # 465
-9.20493694785
objective function at theta- for step # 465
-9.20500998792
objective function at theta+ for step # 470
-9.20500737982
objective function at theta- for step # 470
-9.20493692674
objective function at theta+ for step # 475
-9.2049723228
objective function at theta- for step # 475
-9.20499836294
objective function at theta+ for step # 480
-9.20492283767
objective function at theta- for step # 480
-9.20499965432
objective function at theta+ for step # 485
-9.20492717095
objective function at theta- for step # 485
-9.20495796715
objective function at theta+ for step # 490
-9.20498233952
objective function at theta- for step # 490
-9.20494712296
objective function at theta+ for step # 495
-9.20497798459
objective function at theta- for step # 495
-9.2050053405
Final objective function is: -9.20503789712

OPTIMIZING HAMILTONIAN # 7 AT INTERATOMIC DISTANCE 1.2 ANGSTROM

calibration step # 0 of 5
calibrated SPSA_parameters[0] is 2.02767

objective function at theta+ for step # 310
-9.10296013824
objective function at theta- for step # 310
-9.10300665103
objective function at theta+ for step # 315
-9.10377149472
objective function at theta- for step # 315
-9.10321952482
objective function at theta+ for step # 320
-9.10444153081
objective function at theta- for step # 320
-9.10395648994
objective function at theta+ for step # 325
-9.10413022483
objective function at theta- for step # 325
-9.10461970183
objective function at theta+ for step # 330
-9.10428888175
objective function at theta- for step # 330
-9.10514100602
objective function at theta+ for step # 335
-9.10514238014
objective function at theta- for step # 335
-9.10540034206
objective function at theta+ for step # 340
-9.10571605721
objective function at theta- for step # 340
-9.1056112388
objective function at theta+ for step # 345
-9.10585700647
objective function at theta- for step # 345
-9.10558587093
objective function at theta+ for step # 350
-9.1062785213

objective function at theta+ for step # 155
-9.02046043983
objective function at theta- for step # 155
-9.01973104511
objective function at theta+ for step # 160
-9.02075045759
objective function at theta- for step # 160
-9.0205472928
objective function at theta+ for step # 165
-9.02165609572
objective function at theta- for step # 165
-9.02109375805
objective function at theta+ for step # 170
-9.02170265708
objective function at theta- for step # 170
-9.02253934007
objective function at theta+ for step # 175
-9.02282650912
objective function at theta- for step # 175
-9.02255341832
objective function at theta+ for step # 180
-9.02341095341
objective function at theta- for step # 180
-9.02315281259
objective function at theta+ for step # 185
-9.02386836562
objective function at theta- for step # 185
-9.02384957879
objective function at theta+ for step # 190
-9.02431195894
objective function at theta- for step # 190
-9.02419145925
objective function at theta+ for step # 195
-9.0251510592

calibrated SPSA_parameters[0] is 2.31920021917
objective function at theta+ for step # 0
-8.33932399827
objective function at theta- for step # 0
-8.33949297241
objective function at theta+ for step # 5
-8.23704224303
objective function at theta- for step # 5
-8.23928094249
objective function at theta+ for step # 10
-8.46128219085
objective function at theta- for step # 10
-8.45354584202
objective function at theta+ for step # 15
-8.63627847925
objective function at theta- for step # 15
-8.6389321366
objective function at theta+ for step # 20
-8.78205594849
objective function at theta- for step # 20
-8.78077844345
objective function at theta+ for step # 25
-8.82376250628
objective function at theta- for step # 25
-8.82159866617
objective function at theta+ for step # 30
-8.87825156602
objective function at theta- for step # 30
-8.87692318597
objective function at theta+ for step # 35
-8.89879594968
objective function at theta- for step # 35
-8.89603778294
objective function at theta+ f

objective function at theta+ for step # 350
-8.97196706447
objective function at theta- for step # 350
-8.97195542875
objective function at theta+ for step # 355
-8.97186667769
objective function at theta- for step # 355
-8.97214316688
objective function at theta+ for step # 360
-8.97213708053
objective function at theta- for step # 360
-8.97200591547
objective function at theta+ for step # 365
-8.97213249669
objective function at theta- for step # 365
-8.97210071716
objective function at theta+ for step # 370
-8.97211574896
objective function at theta- for step # 370
-8.97217811898
objective function at theta+ for step # 375
-8.97224490823
objective function at theta- for step # 375
-8.97210100211
objective function at theta+ for step # 380
-8.97222324945
objective function at theta- for step # 380
-8.97224290772
objective function at theta+ for step # 385
-8.97216601077
objective function at theta- for step # 385
-8.97235048211
objective function at theta+ for step # 390
-8.972282523